In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
adata= sc.read_10x_mtx("/Users/shinjinichakraborty/desktop/filtered_feature_bc_matrix")
adata #check adata at this stage
adata.obs #gives list of cells. It is a pandas dataframe
adata.var #gives list of genes. It is a pandas dataframe
adata.X   #sparse matrix combining cell and gene data

#adding columns to the object
adata.obs["cell_label"]="cell_xx"
adata.obs #checking if new column is added to the object

#next step is quality control.This will include filtering out low reads in cells and similar noise! 
#This is different from what cell-ranger does.
#low gene expression, less cells with particular gene expressed and mitochondrial gene

#cells with low gene expression - preprocessing -pp

sc.pp.filter_cells(adata,min_genes=50)
adata #check if adata has filtered genes excluded

#filter low gene expression - preprocessing -pp

sc.pp.filter_genes(adata,min_cells=3)
adata #check if adata has filtered cells excluded

#filter mitochondrial content
adata.obs['percent_mito'] = np.sum(adata[:, adata.var['mt'].values].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
adata.obs #to check if 'percent mito' is added to the object dataframe
sc.pl.violin(adata, 'percent_mito', jitter=0.4) #gives violin plot, giving an idea what threshold can be suitably set
adata = adata[adata.obs['percent_mito'] < 0.3, :] #this is called boolean filtering
adata #check if adata has filtered mitochondrial genes

# Normalization has two steps - preprocessing normalization and log transformation

sc.pp.normalize_total(adata, target_sum=1e4) #normalization
sc.pp.log1p(adata) #log transformation
adata #Anndata object number remains the same, but an additional layer is added when log transformed

# Downstream analysis starts with dimensionality reduction. First step is PCA
# And then, PCA output used as the input for tSNE and UMAP

sc.tl.pca(adata, svd_solver='arpack')

adata #several other layers added

sc.pl.pca(adata, color='CST3')

#Now we perform UMAP
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40) # this computes neighborhood graph required for UMAP
sc.tl.umap(adata)
# Run Louvain clustering
sc.tl.louvain(adata)
# Plot PCA colored by Louvain clusters
sc.pl.pca(adata, color='louvain')

# We can plot gene specific UMAPs as well 

sc.pl.pca_variance_ratio(adata, log=True)
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.paga(adata)
sc.pl.paga(adata, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(adata, init_pos='paga')
sc.tl.umap(adata)
sc.pl.umap(adata, color=['CST3', 'NKG7', 'PPBP'])

